## Projeto 3 - Tratamento de Dados do OpenStreetMap
##### Por Nikolas Thorun

Este projeto se baseia no tratamento de dados de um mapa exportado do OpenStreetMap e consiste em quatro fases: Download e investigação dos dados, tratamento iterativo dos dados, inserção dos dados tratados no banco de dados e buscas no banco de dados.
O mapa escolhido foi o da cidade de Southampton, na Inglaterra, para evitar os percalços de se tentar trabalhar com o UNICODE, já que os mapas de cidades do Brasil têm caracteres especiais como _á, ã, ç_ e etc., que não são suportados pelo *encoding* padrão do Python 2. O arquivo do mapa tem pouco mais de 65 MB descompactado. <br/>
O mapa de Southampton pode ser acessado em: https://www.openstreetmap.org/node/2478628079#map=14/50.9025/-1.4042 <br/>
O banco de dados a ser utilizado é o MongoDB.

In [1]:
#importa bibliotecas necessárias
import xml.etree.cElementTree as ET
import pprint
import re
import codecs
import json

#local do mapa dentro do meu computador
filename = 'C:/Users/Nikolas/Desktop/map/southampton_england.osm'

Utilizando parte do código generosamente concedido por Shannon Bradshaw, aqui é feita a primeira varredura do mapa para a identificação dos tipos de vias mais comuns. Uma vez identificados, os tipos com maior incidência são manualmente inseridos na lista *expected*. Todos os tipos de vias que não estão contidos na lista são printados abaixo.

In [2]:
from collections import defaultdict

street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

expected = ["Avenue", "Bridge", "Buildings", "Centre", "Close", "Court", "Crescent", "Drive", "Gardens", 
            "Grove", "Hill", "Lane", "Mews", "Place", "Road", "Square", "Street", "Terrace", "Walk", "Way"]

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)


def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")


def audit(osmfile):
    osm_file = codecs.open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types


st_types = audit(filename)
pprint.pprint(dict(st_types))

{'387': set(['387']),
 'Broadway': set(['Midanbury Broadway', 'The Broadway']),
 'Cloisters': set(['The Cloisters']),
 'Cottages': set(['Honeysuckle Cottages']),
 'Dell': set(['Holly Dell']),
 'Drove': set(['Coxford Drove', 'York Drove']),
 'East': set(['Bassett Crescent East',
              'Bitterne Road East',
              'Millbrook Road East']),
 'Esplanade': set(['Western Esplanade']),
 'Estate': set(['Belgrave Industrial Estate']),
 'Finches': set(['The Finches']),
 'Firs': set(['The Firs']),
 'Green': set(['Chiltern Green', 'Edwin Jones Green', 'Lulworth Green']),
 'Greenways': set(['Greenways']),
 'High-Rise': set(['Ferry High-Rise']),
 'Holt': set(['Aspen Holt']),
 'House': set(['Beech House']),
 'Loop': set(['Spitfire Loop']),
 'Mayflowers': set(['The Mayflowers']),
 'Meadow': set(['Bassett Meadow']),
 'Mount': set(['The Mount']),
 'North': set(['Manor Road North', 'Osborne Road North']),
 'Parade': set(['Harbour Parade', 'Marine Parade']),
 'Park': set(['Antelope Park', 'P

Aqui percebemos a incidência de alguns nomes errados, como 'road' e 'Raod' e outros que tipos que simplesmente não são tão comuns. Há também a incidência de casos como 'Greenways', 'Saltmead' e 'Redhill', que não possuem tipos, mas podem se desdobrar em vias de mesmo nome e de outro tipo, conforme mostra a figura abaixo.
![Redhill](http://i64.tinypic.com/1hbhaa.png)

Além disso, nomes como 'Royal Crescent Road student re' e 'bluebell road' necessitam de atualizações não apenas no tipo de via, por isso foram inseridos em uma lista de atualização feita unicamente para eles.
Na função abaixo, as vias problemáticas e os tipos de vias problemáticos são identificados e têm seus nomes modificados a partir dos dicionários *mapping* e *road_mapping*. Ao final, são mostrados os nomes antes da atualização e depois da atualização.

In [3]:
def update_name(name, mapping):
    
    m = street_type_re.search(name)
    
    if name in hand_update:
        name = name.replace(name, road_mapping[name])
    elif m:
        street_type = m.group()
        if street_type in mapping:
            name = name.replace(street_type, mapping[street_type])
  
    return name

mapping = { "Raod" : "Road",
            "Rd" : "Road",
            "road" : "Road",
            "Westal" : "West"
            }

road_mapping = { "Royal Crescent Road student re" : "Royal Crescent Road",
                 "Emergency department access" : "Emergency Department Access",
                 "Western Esplanade (corner of Fitzhugh Street)" : "Western Esplanade",
                 "bluebell road" : "Bluebell Road"
                }

hand_update = ["Royal Crescent Road student re", "Emergency department access", 
               "Western Esplanade (corner of Fitzhugh Street)", "bluebell road"]

for st_type, ways in st_types.iteritems():
        for name in ways:
            better_name = update_name(name, mapping)
            print name, "=>", better_name
            

Bassett Meadow => Bassett Meadow
The Polygon => The Polygon
Honeysuckle Cottages => Honeysuckle Cottages
Bassett Crescent West => Bassett Crescent West
Millbrook Road West => Millbrook Road West
Bitterne Road West => Bitterne Road West
Grange Rd => Grange Road
Hythe Rd => Hythe Road
Bellevue Rd => Bellevue Road
Bitterne Road East => Bitterne Road East
Millbrook Road East => Millbrook Road East
Bassett Crescent East => Bassett Crescent East
Shamrock Quay => Shamrock Quay
Town Quay => Town Quay
Royal Crescent Road student re => Royal Crescent Road
Aspen Holt => Aspen Holt
Antelope Park => Antelope Park
Portswood Park => Portswood Park
Holly Dell => Holly Dell
Queensway => Queensway
Emergency department access => Emergency Department Access
Western Esplanade (corner of Fitzhugh Street) => Western Esplanade
Bluebell Raod => Bluebell Road
Shirley Precinct => Shirley Precinct
The Firs => The Firs
Greenways => Greenways
The Cloisters => The Cloisters
The Mount => The Mount
Ferry High-Rise => 

Podemos ver que os tipos de vias incomuns não sofreram mudanças e as vias com erros foram corrigidas, com exceção das vias 'S' e '387', que acredito serem erros de inserção de dados, já que não existem vias com estes nomes na cidade.

O código a seguir transforma os dados tipo XML do arquivo do mapa em tipo JSON, para ser inserido no MongoDB. Nesta fase os nomes atualizados são inseridos em na variável *data*, que será depois lida pelo banco. 

In [4]:
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

CREATED = [ "version", "changeset", "timestamp", "user", "uid"]


def shape_element(element):
    created = {}
    node = {}
    nodes = []
    adress = {}
    posi = []
    if element.tag == "node" or element.tag == "way" :
        node['id'] = element.attrib['id']
        node['visible'] = "true"
        node['type'] = element.tag
        if element.tag == "node" :
            node['pos'] = [float(element.attrib['lat']), float(element.attrib['lon'])]
        created['changeset'] = element.attrib['changeset']
        created['user'] = element.attrib['user']
        created['version'] = element.attrib['version']
        created['uid'] = element.attrib['uid']
        created['timestamp'] = element.attrib['timestamp']
        node['created'] = created
        for tag in element.iter("tag"):
            if tag.attrib['k'] == "addr:housenumber":
                adress['housenumber'] = tag.attrib['v']
            elif  tag.attrib['k'] == "addr:street":
                m = street_type_re.search(tag.attrib['v'])
                if m.group() not in expected:
                    updated_name = update_name(tag.attrib['v'], mapping)
                    adress['street'] = updated_name
                else:
                    adress['street'] = tag.attrib['v'] 
        for nd in element.iter("nd"):
            if nd.attrib['ref']:
                nodes.append(nd.attrib['ref'])
            node['address'] = adress
            node['node_refs'] = nodes
        
        return node
    else:
        return None


def process_map(file_in, pretty = False):
    file_out = "C:/Users/Nikolas/Desktop/southampton_england.json".format(file_in)
    data = []
    with codecs.open(file_out, "w") as fo:
        for _, element in ET.iterparse(file_in):
            el = shape_element(element)
            if el:
                data.append(el)
                if pretty:
                    fo.write(json.dumps(el, indent=2)+"\n")
                else:
                    fo.write(json.dumps(el) + "\n")
    return data

data = process_map(filename, False)

In [5]:
#importa os dados da variável 'data' para o banco de dados
from pymongo import MongoClient
client = MongoClient("mongodb://localhost:27017")
db = client.cities
db.southampton.drop()
db.southampton.insert_many(data)

Com os dados inseridos no banco, podemos começar a fazer as buscas. Abaixo, listo algumas das questões que gostaria de responder:
* Quantos documentos foram inseridos no banco? <br/>
* Quantos desses documentos possuem uma via? <br/>
* Os dados dos tipos 'node' e 'way' foram inseridos corretamente? <br/>
* Quais são os 10 usuários que mais contribuíram com este mapa? Quantas contribuições eles deram? <br/>
* Qual são as 5 ruas que têm maior incidência no mapa? Quantas vezes elas aparecem?

A busca a seguir mostra quantos documentos foram inseridos no banco.

In [6]:
print "Documentos inseridos no banco:"
pprint.pprint(db.southampton.find().count())

Documentos inseridos no banco:
324806


As buscas abaixo mostram quantos documentos são do tipo *way*, *node* e quantos documentos do tipo *way* possuem via.

In [7]:
print "Documentos do tipo node:"
pprint.pprint(db.southampton.find({"type" : "node"}).count())

Documentos do tipo node:
273331


In [8]:
print "Documentos do tipo way:"
pprint.pprint(db.southampton.find({"type" : "way"}).count())

Documentos do tipo way:
51475


In [9]:
print "Documentos do tipo way com nome da rua:"
pprint.pprint(db.southampton.find({"address.street" : {"$exists" : 1}}).count())

Documentos do tipo way com nome da rua:
22781


As buscas abaixo mostram exemplos de documentos tipo *node* e *way*, comprovando que foram inseridos corretamente.
Aqui, a biblioteca **pprint** é importante para facilitar a visualização dos dados.

In [10]:
pprint.pprint(db.southampton.find_one({"type" : "node"}))

{u'_id': ObjectId('595056ce2c38fa0e2c0a9fb0'),
 u'created': {u'changeset': u'8139974',
              u'timestamp': u'2011-05-14T11:45:29Z',
              u'uid': u'260682',
              u'user': u'monxton',
              u'version': u'5'},
 u'id': u'132707',
 u'pos': [50.9454657, -1.4775675],
 u'type': u'node',
 u'visible': u'true'}


In [11]:
pprint.pprint(db.southampton.find_one({"type" : "way", "address.street" : {"$exists" : 1}}))

{u'_id': ObjectId('595056d32c38fa0e2c0ed558'),
 u'address': {u'housenumber': u'Berth 106',
              u'street': u'Herbert Walker Avenue'},
 u'created': {u'changeset': u'35345080',
              u'timestamp': u'2015-11-16T09:22:22Z',
              u'uid': u'1569426',
              u'user': u'Harjit (CabMyRide)',
              u'version': u'9'},
 u'id': u'10517683',
 u'node_refs': [u'90588898',
                u'90588899',
                u'90588900',
                u'90588901',
                u'90588898'],
 u'type': u'way',
 u'visible': u'true'}


A busca abaixo mostra os dez usuários que mais contribuíram para esta parte do mapa e o número de contribuições de cada um.

In [12]:
resultado = db.southampton.aggregate([ 
                                     {"$group" : { "_id" : "$created.user", "count" : {"$sum" : 1}}},
                                     {"$sort" : {"count" : -1}},{"$limit": 10} 
                                     ])

for document in resultado:
    pprint.pprint(document)

{u'_id': u'Chris Baines', u'count': 107158}
{u'_id': u'Harjit (CabMyRide)', u'count': 24971}
{u'_id': u'0123456789', u'count': 23764}
{u'_id': u'Nick Austin', u'count': 17681}
{u'_id': u'pcman1985', u'count': 14158}
{u'_id': u'Deanna Earley', u'count': 13438}
{u'_id': u'Arjan Sahota', u'count': 12948}
{u'_id': u'Kuldip (CabMyRide)', u'count': 9619}
{u'_id': u'Andy Street', u'count': 9189}
{u'_id': u'Harry Cutts', u'count': 6676}


A busca abaixo mostra as cinco vias com maior incidência no mapa e 'None', já que a maior parte dos documentos não possui a variável 'address.street' preenchida.

In [13]:
resultado = db.southampton.aggregate([ 
                                     {"$group" : { "_id" : "$address.street", "count" : {"$sum" : 1}}},
                                     {"$sort" : {"count" : -1}},{"$limit": 6} 
                                     ])

for document in resultado:
    pprint.pprint(document)

{u'_id': None, u'count': 302025}
{u'_id': u'Burgess Road', u'count': 371}
{u'_id': u'Portswood Road', u'count': 349}
{u'_id': u'Winchester Road', u'count': 319}
{u'_id': u'Honeysuckle Road', u'count': 272}
{u'_id': u'Hill Lane', u'count': 250}


#### Bibliografia
* Documentação e tutoriais do Python: https://docs.python.org/2/
* Documentação e tutoriais do MongoDB: https://docs.mongodb.com
* Ajuda para aprender Expressões Regulares: http://regexr.com